# INFO

This notebook will be used for construction and testing purposes while designing model within Kedro framework. 

In [1]:
##############################################################################
# It is recommended to create new virtual environment for each Kedro project #
##############################################################################

# Uncomment and run the line below if your environment does't have
# Kedro or any other dependencies needed.

#! pip install -r requirements.txt
%load_ext kedro.ipython

[11/07/24 22:08:12] INFO     Using                                                                  ]8;id=23650;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=393320;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/k                
                             edro/framework/project/rich_logging.yml' as logging configuration.                    

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=927726;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=660864;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=603614;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=935418;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=770078;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=112661;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/andrejcemeris/Projects/Python3/lending-club.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/07/24 22:08:14] INFO     Kedro project lending_club                                             ]8;id=573114;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=381729;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=853525;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=603780;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[11/07/24 22:08:16] INFO     Registered line magic 'run_viz'                                        ]8;id=4701;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=740666;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\


----

# Baseline model
TODO Remove
As the dataset needs transformation like imputation and normalization, for avoiding data leakage, all transformations will be done within model pipeline and fitting only on training data on model fitting stage. So I'm going to split initial typed dataset to train/test sets and balance train set, ignoring all previously transformed datasets.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import pandas as pd
import logging
from lending_club.pipelines.analysis.nodes import features_eng
from lending_club.pipelines.encode.nodes import _default_status
from imblearn.pipeline import make_pipeline as imb_make_pipeline

logger = logging.getLogger(__name__)

def split_dataset(df: pd.DataFrame, params: dict):
    y = _default_status(df, params)
    X = df
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=params['test_size'],
        random_state=params['random_state']
    )
    return X_train, X_test, y_train, y_test

def train_model(X_train, y_train, regressor, params: dict):
    try:
        regressor.set_params(**params['fit_options']).fit(X_train, y_train)
    except:
        regressor.fit(X_train, y_train)
    return regressor


def model_pipeline(model_options: dict, params: dict):

    # split important features to assign preprocessing steps
    category_feat = [f for f in (params['category'] + [params['emp_len']]) if f in params['model_features']]
    numeric_feat_zero = [f for f in params['fill_zero'] if f in params['model_features']]
    numeric_feat_med = [f for f in params['fill_med'] if f in params['model_features']]

    # transformer to replace missing numeric values by 0
    # and standartize all values
    numeric_feat_zero_transformer = make_pipeline(
        SimpleImputer(strategy='constant', fill_value=0),
        StandardScaler()
    )
    # transformer to replace missing numeric values by median
    numeric_feat_med_transformer = make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler()
    )

    # assemble transformers in preprocessing pipe so it will perform
    # following transformations:
    #   - encode all categorical features to numbers
    #   - fill missing values in specific number features as "0" and standartize them
    #   - fill missing values in specific number features as median and standartize them
    preprocessing = make_column_transformer(
        (OrdinalEncoder(), category_feat),
        (numeric_feat_zero_transformer, numeric_feat_zero),
        (numeric_feat_med_transformer, numeric_feat_med)
    )

    # choose regressor depending on provided model_options
    if model_options['name'] == 'rfc':
        regressor = RandomForestClassifier(**model_options['regressor_options'])
    else:
        if model_options['name'] == 'catboost':
            regressor = CatBoostClassifier(**model_options['regressor_options'])
        else:
            raise Exception("Pipeline accepts only RandomForestClassifier and CatBoostClassifier")

    # assebmle preprocessing pipeline, SMOTE (ibmalance handler) and
    # choosen regressor as the model pipeline
    model = imb_make_pipeline(
        preprocessing,
        SMOTE(random_state=params['random_state']),
        regressor
    )
    return model


In [3]:

# Load parameters
params = catalog.load("parameters")
params['model_features'] = catalog.load("params:model_features")
params['model_options'] = catalog.load("params:baseline_model.model_options")


# catalog.datasets

[11/07/24 22:08:18] INFO     Loading data from parameters (MemoryDataset)...                    ]8;id=319328;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=101612;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from params:model_features (MemoryDataset)...         ]8;id=632873;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=847707;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from params:baseline_model.model_options              ]8;id=786591;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=405144;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [4]:
df = catalog.load("intermediate_lc_dataset")
features_eng(df, params)
X_train, X_test, y_train, y_test = split_dataset(df, params)

                    INFO     Loading data from intermediate_lc_dataset (ParquetDataset)...      ]8;id=545689;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=808126;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [5]:
# Make a model an fit it
model = model_pipeline(params['model_options'], params)
model.fit(X_train, y_train)
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['home_ownership',
                                                   'sub_grade',
                                                   'verification_status']),
                                                 ('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['mths_since_rcnt_il']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('si...
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['open_rv_24m', 'open_acc_6m',
                                                   'max_bal_bc',
                                                   'mths_since_recent_inq',
                                                   'percent_bc_gt_75',
                                                   'bc_open_to_buy',
                                                   'tot_cur_bal',
                                                   'mo_sin_rcnt_rev_tl_op',
                                                   'acc_open_past_24mths'])])),
                ('smote', SMOTE(random_state=42)),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='log_loss',
                                        random_state=42))])

# Evaluation

To evaluate model performance I want to use custom loss function, so I need to calculate actual earning rate to define potential losses in case we refuse in loan, that was mistakenly predicted as default, as well as actual losses for charged off loans, that will be our loss in case if we issue a loan that was mistakenly predicted as non default. 

I assume, that earning rate for non defaulted loans, considering loans that is not fully paid at the moment, is total received amount less than total received principal divided by total received principal. 

For charged off loan, I believe, the actual losses are amount of loan less than total received payments (that includes collections after charges off) plus collection recovery fee (that I believe is our payment to collectors for collection services). Dividing that by this category loan amount we can get actual loss rate for defaulted loans

In [6]:
# Function that returns actual profit/loss rates for non-defartet/defaulted loans
def get_loss_values(df: pd.DataFrame) -> pd.DataFrame:

    # Select columns for profit/loss calculation
    df = df.loc[:, ['loan_amnt', 'loan_status', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee']]

    # Add default status and summarize data
    df['default_status'] = df['loan_status'].str.contains("Charged Off", regex=False, na=False)
    df = df.drop(columns=['loan_status'])
    df=df.groupby(by='default_status').sum()
    df = df.reset_index()


    df['earning/loss'] = (
        # actual earnings rate for non-defaulters
        ((df.total_pymnt - df.total_rec_prncp) / df.total_rec_prncp) * ~df.default_status
        # actual losses rate for defaulters
        + (df.loan_amnt - df.total_pymnt + df.collection_recovery_fee) / df.loan_amnt * df.default_status
        )

    # Select columns: 'default_status', 'earning/loss'
    df = df.loc[:, ['default_status', 'earning/loss']]
    return df.set_index('default_status')

df_loss = get_loss_values(catalog.load('intermediate_lc_clean'))
df_loss

[11/07/24 22:09:35] INFO     Loading data from intermediate_lc_clean (ParquetDataset)...        ]8;id=362177;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=852286;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

,earning/loss
default_status,
False,0.228696
True,0.460871


>These figures will be used in parameters...yml to feed FP_cost and FN_cost to model evaluator

In [7]:
params['FP_cost'] = df_loss['earning/loss'].loc[False]
params['FN_cost'] = df_loss['earning/loss'].loc[True]

In [8]:
def make_rng(start, stop, step):
    return range(start, stop, step)


def evaluate_metrics(model: object, X_true, y_true,
                     params: dict) -> pd.DataFrame:
    y_pred_proba = model.predict_proba(X_true)
    metrics = pd.DataFrame()
    for thresh in make_rng(**params['model_options']['prob_threshold']):
        y_pred = (y_pred_proba[:,1] > (thresh / 100))
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        cur_metrics = pd.DataFrame(
        data={
            'prob_thresh_%': thresh,
            'accuracy'     : accuracy_score(y_true, y_pred),
            'precision'    : precision_score(y_true, y_pred),
            'recall'       : recall_score(y_true, y_pred),
            'f1'           : f1_score(y_true, y_pred),
            'roc_auc'      : roc_auc_score(y_true, y_pred),
            'tn'           : tn,
            'fp'           : fp,
            'fn'           : fn,
            'tp'           : tp,
            'loss'         : params['FP_cost'] * fp + params['FN_cost'] *fn,
        },
        index = [params['model_options']['name']]
        )
        metrics = pd.concat([metrics, cur_metrics], axis=0)
        logger.info(f"The best probability threshold for {params['model_options']['name']} model based on min loss: {metrics[metrics.loss==metrics.loss.min()]['prob_thresh_%'].iloc[0]}")
    return metrics

eval_metr = evaluate_metrics(model, X_test, y_test, params)

eval_metr

,prob_thresh_%,accuracy,precision,recall,f1,roc_auc,tn,fp,fn,tp,loss
rfc,30,0.78045,0.244555,0.381342,0.298002,0.608676,14677,2879,1512,932,1355.251005
rfc,31,0.78920,0.250703,0.364566,0.297099,0.606440,14893,2663,1553,891,1324.748479
rfc,32,0.79640,0.253632,0.342881,0.291580,0.601208,15090,2466,1606,838,1304.121617
rfc,33,0.80320,0.257635,0.324468,0.287215,0.597157,15271,2285,1651,793,1283.466917
rfc,34,0.80825,0.259092,0.306056,0.280623,0.592108,15417,2139,1696,748,1270.816559
rfc,35,0.81500,0.266889,0.294190,0.279875,0.590846,15581,1975,1725,719,1246.675750
rfc,36,0.82090,0.271486,0.276596,0.274017,0.586635,15742,1814,1768,676,1229.673218
rfc,37,0.82590,0.274935,0.259411,0.266947,0.582086,15884,1672,1810,634,1216.555030
rfc,38,0.83135,0.281822,0.245499,0.262410,0.579203,16027,1529,1844,600,1199.521180
rfc,39,0.83555,0.286939,0.232815,0.257059,0.576136,16142,1414,1875,569,1187.508192


In [9]:
print(f"The best probability threshold based on min loss: {eval_metr[eval_metr.loss==eval_metr.loss.min()]['prob_thresh_%'].iloc[0]}")
eval_metr[eval_metr.loss==eval_metr.loss.min()]

The best probability threshold based on min loss: 61


,prob_thresh_%,accuracy,precision,recall,f1,roc_auc,tn,fp,fn,tp,loss
rfc,61,0.87455,0.396825,0.051146,0.090613,0.520162,17366,190,2319,125,1112.21153


In [41]:
%reload_kedro
params['model_options'] = catalog.load("params:candidate_model.model_options")

# Make a model an fit it
model = model_pipeline(params['model_options'], params)
model.fit(X_train, y_train)
model


[11/08/24 05:30:23] INFO     Resolved project path as:                                              ]8;id=299079;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=740850;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/andrejcemeris/Projects/Python3/lending-club.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/08/24 05:30:24] INFO     Kedro project lending_club                                             ]8;id=71902;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=722064;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=431584;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=669987;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=504788;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=593565;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\

                    INFO     Loading data from params:candidate_model.model_options             ]8;id=747381;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=284534;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

0:	learn: 0.6706042	test: 0.6708258	best: 0.6708258 (0)	total: 36.2ms	remaining: 18.1s
100:	learn: 0.2850613	test: 0.2844876	best: 0.2844876 (100)	total: 4.12s	remaining: 16.3s
200:	learn: 0.2374456	test: 0.2386418	best: 0.2386418 (200)	total: 7.57s	remaining: 11.3s
300:	learn: 0.2200728	test: 0.2226986	best: 0.2226986 (300)	total: 11.4s	remaining: 7.53s
400:	learn: 0.2112677	test: 0.2159009	best: 0.2158949 (399)	total: 15.5s	remaining: 3.82s
499:	learn: 0.2072486	test: 0.2142476	best: 0.2142430 (498)	total: 19.3s	remaining: 0us

bestTest = 0.2142429568
bestIteration = 498

Shrink model to first 499 iterations.


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['home_ownership',
                                                   'sub_grade',
                                                   'verification_status']),
                                                 ('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['mths_since_rcnt_il']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('si...
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['open_rv_24m', 'open_acc_6m',
                                                   'max_bal_bc',
                                                   'mths_since_recent_inq',
                                                   'percent_bc_gt_75',
                                                   'bc_open_to_buy',
                                                   'tot_cur_bal',
                                                   'mo_sin_rcnt_rev_tl_op',
                                                   'acc_open_past_24mths'])])),
                ('smote', SMOTE(random_state=42)),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x132e7f760>)])

In [42]:

eval_metr = evaluate_metrics(model, X_test, y_test, params)

eval_metr

,prob_thresh_%,accuracy,precision,recall,f1,roc_auc,tn,fp,fn,tp,loss
catboost,30,0.84205,0.314671,0.248363,0.277613,0.586531,16234,1322,1837,607,1148.955115
catboost,31,0.84655,0.321937,0.231178,0.269112,0.581698,16366,1190,1879,565,1138.123882
catboost,32,0.84990,0.326923,0.215630,0.259862,0.576914,16471,1085,1917,527,1131.623945
catboost,33,0.85340,0.332418,0.198036,0.248205,0.571335,16584,972,1960,484,1125.598797
catboost,34,0.85655,0.338648,0.182488,0.237171,0.566438,16685,871,1998,446,1120.013641
catboost,35,0.85865,0.340550,0.167349,0.224417,0.561118,16764,792,2035,409,1118.998916
catboost,36,0.86135,0.347544,0.153437,0.212887,0.556668,16852,704,2069,375,1114.543319
catboost,37,0.86360,0.355691,0.143208,0.204201,0.553547,16922,634,2094,350,1110.056404
catboost,38,0.86430,0.353261,0.132979,0.193222,0.549544,16961,595,2119,325,1112.659049
catboost,39,0.86595,0.358760,0.123159,0.183369,0.546257,17018,538,2143,301,1110.684305


In [40]:
print(f"The best probability threshold based on min loss: {eval_metr[eval_metr.loss==eval_metr.loss.min()]['prob_thresh_%'].iloc[0]}")
eval_metr[eval_metr.loss==eval_metr.loss.min()]

The best probability threshold based on min loss: 38


,prob_thresh_%,accuracy,precision,recall,f1,roc_auc,tn,fp,fn,tp,loss
catboost,38,0.8646,0.349658,0.125614,0.184828,0.546545,16985,571,2137,307,1115.466031


In [20]:
cb = CatBoostClassifier(**params['model_options']['regressor_options'], cat_features=(params['category']+[params['emp_len']]))